In [1]:
!pip install --quiet feedparser pandas


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.3 MB/s eta 0:00:00


In [2]:
import feedparser
import pandas as pd
import re
from datetime import datetime, timezone

In [3]:
# 1. Definición de los Feeds de Noticias de Aerolíneas
airline_feeds = {
    "United Airlines": "https://www.bing.com/news/search?q=united+airlines&cc=us&qft=interval%3d%228%22&form=PTFTNR&format=rss",
    "US Airways": "https://www.bing.com/news/search?q=US+Airways&cc=us&qft=interval%3d%228%22&form=PTFTNR&format=rss",
    "Southwest": "https://www.bing.com/news/search?q=Southwest&cc=us&qft=interval%3d%229%22&form=PTFTNR&format=rss",
    "Delta Air Lines": "https://www.bing.com/news/search?q=delta+airlines&cc=us&qft=interval%3d%229%22&form=PTFTNR&format=rss",
    "Virgin America": "https://www.bing.com/news/search?q=Virgin+America&cc=us&qft=interval%3d%229%22&form=PTFTNR&format=rss",
    "JetBlue": "https://www.bing.com/news/search?q=JetBlue&cc=us&qft=interval%3d%228%22&form=PTFTNR&format=rss",
}


In [4]:
# 2. Obtención de Datos desde los Feeds RSS
records = []
for airline, rss_url in airline_feeds.items():
    feed = feedparser.parse(rss_url)
    for item in feed.entries:
        records.append({
            "airline": airline,
            "title": item.title,
            "published_at": datetime(*item.published_parsed[:6], tzinfo=timezone.utc).astimezone().isoformat(),
            "url": item.link
        })

df = pd.DataFrame(records)

In [5]:
# 3. Limpieza y Procesamiento de Datos

# 1. Eliminar duplicados y filas sin información esencial
df.drop_duplicates(subset=["airline", "title"], inplace=True)
df.dropna(subset=["title", "url"], inplace=True)

# 2. Función para limpieza ligera conservando mayúsculas, emojis y puntuación básica
import re
import pandas as pd

def limpiar_texto_suave(texto):
    """
    Limpieza ligera para títulos:
    - Elimina URLs.
    - Elimina caracteres de control o invisibles.
    - Normaliza espacios múltiples.
    - Conserva mayúsculas, emojis, puntuación y caracteres especiales relevantes.
    """
    if not isinstance(texto, str):
        return ""

    # Eliminar URLs
    texto = re.sub(r"http\S+|www\S+", "", texto)

    # Eliminar caracteres de control (no imprimibles)
    texto = re.sub(r"[\x00-\x1f\x7f-\x9f]", "", texto)

    # Normalizar espacios múltiples
    texto = re.sub(r"\s+", " ", texto).strip()

    return texto

# 3. Aplicar limpieza y crear columna 'clean_title'
df["clean_title"] = df["title"].apply(limpiar_texto_suave)

# 4. Filtrar títulos con longitud mínima (e.g., > 20 caracteres sin recortar demasiado el dataset)
df = df[df["clean_title"].str.len() > 20]

# 5. Convertir fechas y ordenar
df["published_at"] = pd.to_datetime(df["published_at"], errors="coerce")
df = df.dropna(subset=["published_at"])  # por si alguna fecha no es válida
df.sort_values("published_at", ascending=False, inplace=True)



In [6]:
# 4. Guardar los Datos Limpios en Archivos CSV
# Reordenar columnas para mayor claridad en el CSV
columnas_ordenadas = ["airline", "published_at", "title", "clean_title", "url"]
df = df[columnas_ordenadas]

In [7]:
# Guardar un archivo por cada aerolínea
for aerolinea in df["airline"].unique():
    df_aerolinea = df[df["airline"] == aerolinea]
    nombre_archivo = f"{aerolinea.lower().replace(' ', '_')}.csv"
    df_aerolinea.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
    print(f"✔️ Archivo guardado: {nombre_archivo}")

✔️ Archivo guardado: southwest.csv
✔️ Archivo guardado: delta_air_lines.csv
✔️ Archivo guardado: jetblue.csv
✔️ Archivo guardado: us_airways.csv
✔️ Archivo guardado: united_airlines.csv
✔️ Archivo guardado: virgin_america.csv


In [8]:
from google.colab import files
#files.download("delta_air_lines.csv")
#files.download("southwest.csv")
#files.download("united_airlines.csv")
files.download("jetblue.csv")
#files.download("virgin_america.csv")
#files.download("us_airways.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>